# Download

https://mapreader.readthedocs.io/en/latest/using-mapreader/step-by-step-guide/1-download.html

In [ ]:
from mapreader import SheetDownloader

my_ts = SheetDownloader(
    metadata_path="../../NLS_metadata/metadata_OS_One_Inch_GB_WFS_light.json",
    download_url="https://mapseries-tilesets.s3.amazonaws.com/1inch_2nd_ed/{z}/{x}/{y}.png",
)

In [ ]:
my_ts.extract_wfs_id_nos()
my_ts.plot_all_metadata_on_map(map_extent="uk", add_id=False)

In [ ]:
my_ts.get_minmax_latlon()

In [ ]:
my_ts.extract_published_dates()
my_ts.metadata["published_date"].hist(bins=30, edgecolor="k")


## Query maps using a list of lats/lons

In [5]:
my_ts.query_map_sheets_by_coordinates((-4.33, 55.90))
my_ts.query_map_sheets_by_coordinates((-3.25, 51.93), append=True)

In [ ]:
my_ts.print_found_queries()

In [ ]:
my_ts.plot_queries_on_map(map_extent="uk")

## Download map tiles

In [ ]:
my_ts.get_grid_bb(zoom_level=14)
my_ts.download_map_sheets_by_queries(path_save="../../maps", force=True)

# Load

https://mapreader.readthedocs.io/en/latest/using-mapreader/step-by-step-guide/2-load.html

## Load in downloaded maps

In [9]:
from mapreader import loader

In [ ]:
my_files = loader("../../maps/*png")

In [ ]:
# len() shows the total number of images currently read (or sliced, see below)
print(f"Number of images: {len(my_files)}")

In [ ]:
print(my_files)

In [ ]:
my_files.add_metadata("../../maps/metadata.csv")

In [14]:
parent_list = my_files.list_parents()

## Patchify maps

In [ ]:
my_files.patchify_all(patch_size=1000, path_save="../../patches_1000_pixel")  # in pixels

In [ ]:
my_files.show_sample(num_samples=6, tree_level="patch")

In [ ]:
my_files.show_patches(
    parent_id=parent_list[0],
    figsize=(15, 15)
)

In [ ]:
my_files.explore_patches(
    parent_id=parent_list[0],
    xyz_url="https://mapseries-tilesets.s3.amazonaws.com/1inch_2nd_ed/{z}/{x}/{y}.png"
)

## Add coordinate increments for text spotting

In [ ]:
my_files.add_coord_increments()

In [ ]:
parent_df, patch_df = my_files.convert_images(save=True)

In [ ]:
parent_df.head()

In [ ]:
patch_df.head()

In [23]:
patch_list = my_files.list_patches()

## Detect text

Here, we show how to load an already trained/fine-tuned text detection model and run the model inference on your patches. DPText-DETR is a text detection framework so it produces bounding boxes and scores only (but not OCR text outputs).

### Set up the DPText-DETR runner

In [ ]:
import os

DPTEXT_PATH = os.getenv("DPTEXT_PATH")
DPTEXT_MODEL_PATH = os.getenv("DPTEXT_MODEL_PATH")

In [24]:
from mapreader import DPTextDETRRunner

In [25]:
# change these to your own paths, see the README for more details on how to get these
cfg_file = f"{DPTEXT_PATH}/configs/DPText_DETR/ArT/R_50_poly.yaml"
weights_file = DPTEXT_MODEL_PATH

In [ ]:
my_runner = DPTextDETRRunner(
    patch_df[:10], # only the first 10 to save time
    parent_df,
    cfg_file = cfg_file,
    weights_file = weights_file,
)

### Run on all patches in the patch dataframe

Note: we've only loaded the first 10 rows of our patch dataframe to save time

In [ ]:
patch_predictions = my_runner.run_all(return_dataframe=True)

In [ ]:
patch_predictions.head()

In [ ]:
my_runner.show_predictions(
    patch_list[0],
    figsize=(15, 15),
    border_color="r",
    text_color="b",
    )

### Scale up to parent images

In [30]:
parent_predictions = my_runner.convert_to_parent_pixel_bounds(return_dataframe=True)

In [ ]:
parent_predictions.head()

In [ ]:
my_runner.show_predictions(
    parent_list[0],
    figsize=(15, 15),
    border_color="r",
    text_color="b",
    )

### Convert pixel bounds to coordinates

In [33]:
geo_predictions = my_runner.convert_to_coords(return_dataframe=True)

Saving these outputs will give you a geojson file you can load into a GIS software.

In [34]:
my_runner.save_to_geojson("./example_outputs.geojson")